# Test Dickens
This notebook replicates the functionality of the `test_ollama.py` script.

In [1]:
import os
from lightrag import LightRAG, QueryParam
from lightrag.llm.openai import gpt_4o_mini_complete, gpt_4o_complete, openai_embed
from lightrag.llm.ollama import ollama_model_complete, ollama_embed
from lightrag.llm.siliconcloud import siliconcloud_embedding
from lightrag.utils import EmbeddingFunc
import asyncio
from lightrag import LightRAG, QueryParam
from lightrag.llm.openai import openai_complete_if_cache
from lightrag.llm.siliconcloud import siliconcloud_embedding
from lightrag.utils import EmbeddingFunc
import numpy as np


# Uncomment the below two lines if running in a jupyter notebook to handle the async nature of rag.insert()
import nest_asyncio
nest_asyncio.apply()

Looking in indexes: https://mirrors.cloud.tencent.com/pypi/simple


ERROR: Could not find a version that satisfies the requirement lmdeploy (from versions: none)
ERROR: No matching distribution found for lmdeploy


In [2]:
WORKING_DIR = "./dickens"

if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

# 在导入部分添加
from dotenv import load_dotenv

# 在代码的开头加载环境变量
load_dotenv()

True

In [11]:
async def llm_model_func(
    prompt, system_prompt=None, history_messages=[], keyword_extraction=False, **kwargs
) -> str:
    return await openai_complete_if_cache(
        "Qwen/Qwen2.5-7B-Instruct",
        prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        api_key=os.getenv("SILICONFLOW_API_KEY"),
        base_url="https://api.siliconflow.cn/v1/",
        **kwargs,
    )


async def embedding_func(texts: list[str]) -> np.ndarray:
    return await siliconcloud_embedding(
        texts,
        model="netease-youdao/bce-embedding-base_v1",
        api_key=os.getenv("SILICONFLOW_API_KEY"),
        max_token_size=512,
    )



In [ ]:
# function test
async def test_funcs():
    result = await llm_model_func("How are you?")
    print("llm_model_func: ", result)

    result = await embedding_func(["How are you?"])
    print("embedding_func: ", result)


asyncio.run(test_funcs())


In [ ]:
rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=llm_model_func,
    embedding_func=EmbeddingFunc(
        embedding_dim=768, max_token_size=512, func=embedding_func
    ),
)


In [3]:
# Choose the model here
model_choice = "ollama"  # Options: "ollama", "openai"

llm_model_func = ollama_model_complete
embedding_func = EmbeddingFunc(
    embedding_dim=1024,
    max_token_size=8192,
    func=lambda texts: ollama_embed(
        texts,
        embed_model="bge-m3:latest"
    )
)
rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=llm_model_func, 
    llm_model_name='qwen2.5:7b-instruct-q4_K_M',
    embedding_func=embedding_func,
    llm_model_max_token_size=32768,
    llm_model_kwargs={"host": "http://localhost:11434", "options": {"num_ctx": 32768}},

)

/Users/llp/miniforge3/envs/lightrag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:nano-vectordb:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': './dickens/vdb_entities.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': './dickens/vdb_relationships.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': './dickens/vdb_chunks.json'} 0 data
INFO:lightrag:Loaded document status storage with 0 records


In [4]:
with open("./book.txt") as f:
    rag.insert(f.read())

INFO:lightrag:Stored 1 new unique documents
INFO:lightrag:Number of batches to process: 1.
INFO:lightrag:Inserting 12 vectors to chunks
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached missed(mode:default type:extract)
INFO:lightrag:Non-embedding cached m

In [5]:
# Perform naive search
print("naive.....................")
print(rag.query("What are the top themes in this story?", param=QueryParam(mode="naive")))

INFO:lightrag:Non-embedding cached missed(mode:naive type:query)


naive.....................
Sorry, I'm not able to provide an answer to that question.[no-context]


In [6]:
# Perform local search
print("local.....................")
print(rag.query("What are the top themes in this story?", param=QueryParam(mode="local")))

INFO:lightrag:Non-embedding cached missed(mode:local type:query)
INFO:lightrag:Non-embedding cached missed(mode:local type:keywords)


local.....................


INFO:lightrag:Query nodes: Plot elements, Character development, Setting, Conflict, top_k: 60, cosine: 0.2
INFO:lightrag:Local query uses 60 entites, 65 relations, 0 chunks


The story "A Christmas Carol" by Charles Dickens explores several key themes that resonate throughout the narrative:

1. **Redemption and Transformation**: The central theme revolves around Scrooge's transformation from a miserly, bitter old man to a kind, generous person who values family and charity. This transformation is facilitated by his encounters with the three spirits: the Ghost of Christmas Past, Present, and Yet to Come.

2. **Moral Guidance Through Visions**: The spirits guide Scrooge through his past, present, and future, offering moral lessons that challenge his behavior and influence his character development. These visions reveal the consequences of his actions and the importance of change.

3. **The Spirit of Christmas**: The story emphasizes the true spirit of Christmas, which includes giving, charity, and family bonds. Scrooge's experiences during the visits from the spirits highlight these values and their impact on society.

4. **Social Class and Inequality**: Dick

In [8]:
# Perform global search
print("global.....................")
print(rag.query("What are the top themes in this story?", param=QueryParam(mode="global")))

INFO:lightrag:Non-embedding cached hit(mode:global type:query)


global.....................
The story of "A Christmas Carol" by Charles Dickens is rich with thematic elements that resonate deeply within its narrative. The primary themes include transformation, redemption, and the spirit of Christmas.

### Transformation and Redemption
One of the most significant themes in "A Christmas Carol" is the profound personal transformation experienced by Ebenezer Scrooge. Initially a bitter, miserly old man who despises Christmas and his fellow human beings, Scrooge undergoes a dramatic change after being visited by the three spirits—ghosts of Christmas Past, Present, and Yet to Come. These spirits reveal to him not only his past mistakes but also the dire consequences of his current behavior. Through this journey, Scrooge comes to understand the value of generosity, compassion, and family, ultimately leading to a complete transformation from a miser to a kind and generous man.

### The Spirit of Christmas
The spirit of Christmas is another crucial theme th

In [9]:
# Perform hybrid search
print("hybrid.....................")
print(rag.query("What are the top themes in this story?", param=QueryParam(mode="hybrid")))

INFO:lightrag:Non-embedding cached missed(mode:hybrid type:query)
INFO:lightrag:Non-embedding cached missed(mode:hybrid type:keywords)


hybrid.....................


INFO:lightrag:Query nodes: Narrative elements, Plot development, Character arcs, Symbolism, top_k: 60, cosine: 0.2
INFO:lightrag:Query edges: Top themes, Story analysis, top_k: 60, cosine: 0.2
INFO:lightrag:Global query uses 62 entites, 60 relations, 0 chunks
INFO:lightrag:Local query uses 60 entites, 66 relations, 0 chunks


The story of "A Christmas Carol" by Charles Dickens is rich with several prominent themes that resonate throughout the narrative:

1. **Redemption and Transformation**: One of the most central themes is the transformative journey of Ebenezer Scrooge. Initially portrayed as a bitter, miserly old man, Scrooge undergoes a significant change after being visited by the three spirits of Christmas past, present, and yet to come. These visits reveal his misdeeds and the potential consequences if he does not alter his ways.

2. **The Spirit of Christmas**: The novel emphasizes the true spirit of Christmas, which transcends mere materialism and commercialization. Scrooge learns that generosity, compassion, and kindness are essential virtues that should be practiced throughout the year, not just during the festive season.

3. **Class Differences and Social Inequality**: The story starkly contrasts the wealthy Scrooge with his poor employees and neighbors, such as Bob Cratchit and his family. This

In [10]:
# Perform mix search (Knowledge Graph + Vector Retrieval)
print("mix.....................")
print(rag.query("What are the top themes in this story?", param=QueryParam(mode="mix")))

INFO:lightrag:Non-embedding cached missed(mode:mix type:query)
INFO:lightrag:Non-embedding cached missed(mode:mix type:keywords)


mix.....................


INFO:lightrag:Query edges: Top themes, Story analysis, top_k: 60, cosine: 0.2
INFO:lightrag:Global query uses 62 entites, 60 relations, 0 chunks


The story of "A Christmas Carol" by Charles Dickens is rich with several prominent themes that resonate through its narrative. Here are the key themes:

### Redemption and Transformation
One of the most central themes is the process of redemption and transformation, embodied by the character of Ebenezer Scrooge. Initially a bitter and miserly old man who is skeptical about the value of Christmas and charitable acts, Scrooge undergoes a profound change after being visited by three spirits (the Ghosts of Christmas Past, Present, and Yet to Come). These spirits reveal his past mistakes, current situation, and future consequences if he does not alter his ways. This journey leads him from miserliness to generosity and kindness.

### The Spirit of Christmas
The story emphasizes the spirit of Christmas, encompassing themes such as generosity, community, and joy. Through interactions with family members like Tiny Tim Cratchit and friends like Fred, Scrooge learns that true happiness comes not 